In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"
canon_df = pd.read_csv(file_path)

In [4]:
canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

In [5]:
cols = ['ImageFilePath', 'Genus']
df = canon_df[cols]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax


In [6]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")

tokenizer = open_clip.get_tokenizer("ViT-B-32")

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

for param in model.parameters() :
    param.requires_grad = False


In [7]:
def extract_clip_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()
    

In [8]:
X = np.vstack([extract_clip_features(img) for img in df["ImageFilePath"]])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["Genus"])


df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Extracted feature shape: (1622, 512)


In [9]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128, 32), activation='logistic', alpha=1e-3, learning_rate='adaptive', max_iter=250, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 86.46% | Prec: 86.46% | Rec: 86.46% | F1: 86.43% | Bal Acc: 77.08% | MCC: 0.7175
LogisticRegression        | Acc: 97.85% | Prec: 97.87% | Rec: 97.85% | F1: 97.85% | Bal Acc: 98.39% | MCC: 0.9552
SVMLinear                 | Acc: 97.85% | Prec: 97.88% | Rec: 97.85% | F1: 97.85% | Bal Acc: 98.56% | MCC: 0.9555
SVMPolynomial             | Acc: 72.92% | Prec: 78.83% | Rec: 72.92% | F1: 67.24% | Bal Acc: 47.07% | MCC: 0.3966
SVMRadialBasis            | Acc: 96.31% | Prec: 96.38% | Rec: 96.31% | F1: 96.03% | Bal Acc: 81.88% | MCC: 0.9220
NearestNeighbor           | Acc: 91.08% | Prec: 91.29% | Rec: 91.08% | F1: 90.55% | Bal Acc: 72.77% | MCC: 0.8096
RandomForest              | Acc: 89.85% | Prec: 86.50% | Rec: 89.85% | F1: 88.13% | Bal Acc: 61.32% | MCC: 0.7810
MLP_Baseline              | Acc: 98.15% | Prec: 98.17% | Rec: 98.15% | F1: 98.16% | Bal Acc: 98.55% | MCC: 0.9615


In [15]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [16]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/12.CLIP-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/12.CLIP-linear-probing-genus-metrics.csv", index=False)